## This file uses APIs to gather candlestick data from different exchanges, and converts them into the correct data format which backtrader uses.

In [1]:
# import required libraries

# for handling data retrieval from Binance Exchange and yahoo finance
import binance
import yfinance as yf

# for handling requests and exports if needed
import requests
import csv 

# utility libraries we may need to manipulate data
import time
import dateparser
import pytz
import os
import pandas as pd

from pathlib import Path
from datetime import datetime
from binance.client import Client


ModuleNotFoundError: No module named 'yfinance'

# This section of code gathers data from binance api 

In [ ]:

# This is for checking the library location
# print(binance.__file__)

def date_to_milliseconds(date_str):
    """
    
    Function to convert date to milliseconds

    If using offset strings add "UTC" to date string e.g. "now UTC", "11 hours ago UTC"
    See dateparse docs for formats http://dateparser.readthedocs.io/en/latest/

    params:
       * date_str: date in readable format, i.e. "January 01, 2018", "11 hours ago UTC", "now UTC"
                   This parameter must be of a string data  type
    
    """
    # Keep the  epoch value in GMT
    epoch = datetime.utcfromtimestamp(0).replace(tzinfo=pytz.timezone('GMT'))
    
    # parse our date string
    d = dateparser.parse(date_str)
    
    # if the date is not timezone aware apply GMT timezone
    if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
        d = d.replace(tzinfo=pytz.timezone('GMT'))

    # return the difference in time
    return int((d - epoch).total_seconds() * 1000.0)


def interval_to_milliseconds(interval):
    """
    
    Function to convert interval values to milliseconds

    params:
       * interval: interval string 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w
         

    :return:
         None if unit not one of m, h, d or w
         None if string not in correct format
         int value of interval in milliseconds
         
    """
    ms = None
    
    # define the measurement units
    seconds_per_unit = {
        "m": 60,
        "h": 60 * 60,
        "d": 24 * 60 * 60,
        "w": 7 * 24 * 60 * 60
    }

    # return the millisecond value only if
    # the units are one of the define units of measurements
    unit = interval[-1]
    if unit in seconds_per_unit:
        try:
            ms = int(interval[:-1]) * seconds_per_unit[unit] * 1000
        except ValueError:
            pass
    return ms



def get_historical_klines(symbol, interval, start_str, end_str=None):
    """
    Function to retrieve historical data from Binance using Klines
    
    Since Binance only offers data retrievals to a maximum limit of a data set specified, 
    the purpose of this function is to loop through each data range and iterate through them 
    until the limit is reached then retrieve the next block until we have all data in the desired date range
    we then append each retrieval block into a single array.

    params:
    
      * symbol:    Name of symbol pair e.g BNBBTC
      * interval:  This is the interval of the data ie: 30min, 60min, 1Day, etc
      * start_str: The start date string from when we we would like to retrieve the data from
      * end_str:   The end date string when we want the data to finish
      
      API Sample Response Data:

            [
                [
                    1499040000000,      # Open time
                    "0.01634790",       # Open
                    "0.80000000",       # High
                    "0.01575800",       # Low
                    "0.01577100",       # Close
                    "148976.11427815",  # Volume
                    1499644799999,      # Close time
                    "2434.19055334",    # Quote asset volume
                    308,                # Number of trades
                    "1756.87402397",    # Taker buy base asset volume
                    "28.46694368",      # Taker buy quote asset volume
                    "17928899.62484339" # Can be ignored
                ]
            ]      


    """
    
    # create the Binance client, no need for api key
    # No API key is required to retrieve the quote tickers
    client = Client("", "")

    # init our list
    output_data = []

    # setup the max limit
    limit = 500

    # convert interval to useful value in seconds
    timeframe = interval_to_milliseconds(interval)

    # convert our date strings to milliseconds
    start_ts = date_to_milliseconds(start_str)

    # if an end time was passed convert it
    end_ts = None
    if end_str:
        end_ts = date_to_milliseconds(end_str)

    idx = 0
    # Allow start time to be before list date 
    # to determine if the symbol was listed at the date range
    symbol_existed = False
    
    # iterate through each date block (for each 500 data points)
    # collate the data until we have all our data sets for the date range
    while start_ts < end_ts :
        # fetch the klines from start_ts up to max 500 entries or the end_ts if set
        temp_data = client.get_klines(
            symbol=symbol,
            interval=interval,
            limit=limit,
            startTime=start_ts,
            endTime=end_ts
        )

        # handle the case where our start date is before the symbol pair listed on Binance
        # if there is no data in temp_data then the symbol did not exist during this period
        if not symbol_existed and len(temp_data):
            symbol_existed = True

        if symbol_existed:
            # append this loops data to our output data
            # if the symbol exists, we want to continue to append the data into an array
            # until we reach then end of our specified end date
            output_data += temp_data

            # update our start timestamp using the last value in the array and add the interval timeframe
            # we need to continue to iterate for more data once we have reached our request limit 
            # until we have all the requested data
            start_ts = temp_data[len(temp_data) - 1][0] + timeframe
        else:
            # it wasn't listed yet, increment our start datez
            start_ts += timeframe

        idx += 1
        # check if we received less than the required limit and exit the loop
        if len(temp_data) < limit:
            # exit the while loop
            break

        # sleep after every 3rd call to be kind to the API
        # this is to ensure the API does not block our calls due to API call limits
        if idx % 3 == 0:
            time.sleep(1)
        
    # check for errors    
    if(len(output_data) == 0):
        return ("error: no data exists for this data range - the requested symbol may not have existed during this time")
    else:
        return output_data


    
def get_binance_data(symbol, interval, start_date, end_date):
    """
    Function to retrieve the price data from Binance for a particular data range
    
    This function will retrieve assets prices from Binance for a certain date range and write to a csv file
    The output file produced from this extraction will be in the followign format:
    
       <symbol>.csv
    
    params:
    
       * start_date : Start date to retrieve the data
       * end_date   : The date the data should end
       * symbol     : The symbol to retrieve the data for
       * interval   : The interval of the data eg: 5min, 30min, 1day 
    
    
    
    The format of the csv file will contain the following fields:
    
        Date          : The date for the quoted price given and must be in the format of: ('%Y-%m-%d %H:%M:%S')
                        For validity of the dates, this date will be adjusted to the GMT timezone 
        
        Open          : The price which the particular asset opened at on the day
        
        High          : The highest price reached on the day
        
        Low           : The lowest price reached on the day
        
        Close         : The last recorded price for the day 
        
        Open Interest : This value will always be zero and does not impact strategy outcome
                        It is required by the BackTrader library to simulate strategy output results
    
    ** === Note: The output from the csv file will not contain any headers as it is not required by BackTrader === **
    
    """
    
    # retrieve historical prices using Binance's klines function
    klines = get_historical_klines(symbol, interval, start_date, end_date)
    
    # export location
    export_file = f"./data/{symbol}.csv"
    
    if "error" in klines:
        print("error in get_binance_data(): no data found in the specified date range")
        return;
    else:
        # if we have data within this date range for the specified asset, we need to format the data 
        # retrieved from Binance and format it the specification as above
        
        # create a list to store the collection of all prices
        # for the requested date range
        symbol_history = []
        
        for item in klines:
            
            # converting the date to GMT Zone and format it accordingly..
            # for zone definitions, refer to: https://en.wikipedia.org/wiki/List_of_tz_database_time_zones
            # also note: javascript uses milliseconds internally, in a unix system the unit of measure is seconds so we may need to x1000 for correction
            # and since timestamps/epoch are in milliseconds and datetime are in seconds, a division of 1000 is required to convert from ms to s
            date = datetime.fromtimestamp(int(item[0]/1000), pytz.timezone("Etc/GMT0")).strftime('%Y-%m-%d %H:%M:%S')
            
            # extract other required fields
            open_price  = item[1]
            high_price  = item[2]
            low_price   = item[3]
            close_price = item[4]
            volume      = item[5]
            open_int    = 0
            
            # rebuild the array for each item in the Binance API response:
            quote_row = [date, open_price, high_price, low_price, close_price, volume, open_int]
            
            # for each quote, add to our symbol history list
            symbol_history.append(quote_row)
            
        with open(export_file, 'w', encoding='UTF8', newline='') as f:
            writer = csv.writer(f)

            # no header required so let's just write the rows...
            # write multiple rows
            writer.writerows(symbol_history)
        
    print(f"historical prices for {symbol} has been exported to {export_file}")

            

# extract the data for BTCUSDT from the Binance exchange..
get_binance_data("BTCUSDT", Client.KLINE_INTERVAL_1DAY,"01 Sept, 2017", "27 Sept, 2021")



# This section gathers data from yahoo finance api for SP500

In [ ]:
GSPC = yf.Ticker("^GSPC")

In [ ]:
SP500 = GSPC.history(interval="1d", start="2016-01-01", end="2021-09-28")
SP500

In [ ]:
SP500 = SP500.drop('Stock Splits', axis=1)
SP500 = SP500.drop(SP500.index[0])

In [ ]:
SP500.index = SP500.index.strftime('%Y-%m-%d %H:%M:%S')
SP500

In [ ]:
SP500.to_csv(r'./data/SP500_Data.csv')

# This section gets transforms the data gathered from yahoo finance website in usable data

In [ ]:
csv_path = Path("data/gold_data_raw.csv")

In [ ]:
gold_data = pd.read_csv((csv_path) , index_col = "Date")
gold_data = gold_data.sort_index(ascending=True)
gold_data.index = pd.to_datetime(gold_data.index, format='%Y-%m-%d %H:%M:%S') + pd.Timedelta(seconds=1)

In [ ]:
gold_data.head()

In [ ]:
gold_data['Volume'] = gold_data['Volume'].fillna(0)
gold_data['Volume'].isnull().values.any()

In [ ]:
gold_data.to_csv(r'./data/gold_data.csv')